In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import time
from fractions import Fraction
import random

In [2]:
def generate_matrix(n):
    matrix = []
    for i in range(n):
        row = [Fraction(0) for _ in range(n)]
        if i == 0:
            row[0], row[1] = Fraction(1), Fraction(1,2)
        elif i == n-1:
            row[n-2], row[n-1] = Fraction(1,n), Fraction(1)
        else:
            row[i-1], row[i], row[i+1] = Fraction(1,i+1), Fraction(2), Fraction(1,i+2)
        matrix.append(row)
    return matrix


def generate_x(n):
    random.seed(42)
    return [Fraction(random.randint(-1, 0)) for _ in range(n)]

In [3]:
def jacobi_method(A, b, p):
    n = A.shape[0]
    x = np.zeros(n)

    norm1, norm2 = 2, 2 

    i = 1
    data = {'i': [], 'norm1': [], 'norm2':[]}
    while norm1 > p or norm2 > p:
        new_x = np.array(
            [1/A[i,i] * (b[i] - sum(A[i,j]*x[j] for j in range(n) if j!=i))
            for i in range(n)]
        )
        norm1 = np.linalg.norm(x - new_x)
        norm2 = np.linalg.norm(A @ new_x - b) / np.linalg.norm(b)
        x = new_x

        data['i'].append(i)
        data['norm1'].append(norm1)
        data['norm2'].append(norm2)
        i += 1
    return x, data

In [57]:
n = 100
A, x_correct = generate_matrix(n), generate_x(n)
b = [0 for _ in range(n)]

for i in range(n):
    for j in range(n):
        b[i] += A[i][j] * x_correct[j]

x_correct = np.array(x_correct, dtype='float64')
A, b = np.array(A, dtype='float64'), np.array(b, dtype='float64')

x, data = jacobi_method(A, b, 1e-4)
df = pd.DataFrame(data)
df = df.rename(columns={'i': 'i', 'norm1': '$||x^t - x^{t-1}||$', 'norm2': '$||Ax^{t+1} - b||/||b||$'})
print(df.to_latex())

\begin{tabular}{lrrr}
\toprule
 & i & $||x^t - x^{t-1}||$ & $||Ax^{t+1} - b||/||b||$ \\
\midrule
0 & 1 & 7.808634 & 0.099824 \\
1 & 2 & 0.940120 & 0.034781 \\
2 & 3 & 0.324603 & 0.012783 \\
3 & 4 & 0.135151 & 0.005293 \\
4 & 5 & 0.050265 & 0.001977 \\
5 & 6 & 0.021182 & 0.000825 \\
6 & 7 & 0.007857 & 0.000308 \\
7 & 8 & 0.003310 & 0.000129 \\
8 & 9 & 0.001227 & 0.000048 \\
9 & 10 & 0.000517 & 0.000020 \\
10 & 11 & 0.000192 & 0.000008 \\
11 & 12 & 0.000081 & 0.000003 \\
\bottomrule
\end{tabular}



In [67]:
for precision in [1e-2, 1e-4, 1e-4, 1e-8, 1e-10]:
    x, data = jacobi_method(A, b, precision)
    df = pd.DataFrame(data)
    df = df.rename(columns={'i': 'i', 'norm1': '$||x^t - x^{t-1}||$', 'norm2': '$||Ax^{t+1} - b||/||b||$'})
    tabular = df.to_latex(index=False)
    print(rf"""
    \begin{{table}}
    \centering
    \caption{{Wyniki dla metody Jacobiego z precyzją {str(precision)}. }}
    {tabular}
    \end{{table}}
    """)




    \begin{table}
    \centering
    \caption{Wyniki dla metody Jacobiego z precyzją 0.01. }
    \begin{tabular}{rrr}
\toprule
i & $||x^t - x^{t-1}||$ & $||Ax^{t+1} - b||/||b||$ \\
\midrule
1 & 7.808634 & 0.099824 \\
2 & 0.940120 & 0.034781 \\
3 & 0.324603 & 0.012783 \\
4 & 0.135151 & 0.005293 \\
5 & 0.050265 & 0.001977 \\
6 & 0.021182 & 0.000825 \\
7 & 0.007857 & 0.000308 \\
\bottomrule
\end{tabular}

    \end{table}
    

    \begin{table}
    \centering
    \caption{Wyniki dla metody Jacobiego z precyzją 0.0001. }
    \begin{tabular}{rrr}
\toprule
i & $||x^t - x^{t-1}||$ & $||Ax^{t+1} - b||/||b||$ \\
\midrule
1 & 7.808634 & 0.099824 \\
2 & 0.940120 & 0.034781 \\
3 & 0.324603 & 0.012783 \\
4 & 0.135151 & 0.005293 \\
5 & 0.050265 & 0.001977 \\
6 & 0.021182 & 0.000825 \\
7 & 0.007857 & 0.000308 \\
8 & 0.003310 & 0.000129 \\
9 & 0.001227 & 0.000048 \\
10 & 0.000517 & 0.000020 \\
11 & 0.000192 & 0.000008 \\
12 & 0.000081 & 0.000003 \\
\bottomrule
\end{tabular}

    \end{table}
    

   

In [58]:
def chebyshev(A, b, l_min, l_max, precision):
    d = (l_max + l_min) / 2
    c = (l_max - l_min)/ 2

    x = np.zeros(A.shape[0])
    r = b - A @ x
    norm1, norm2 = 2,2


    i = 1
    data = {'i': [], 'norm1': [], 'norm2':[]}
    while norm1 > precision or norm2 > precision:
        z = r

        if i == 1:
            p = z
            alpha = 1/d
        elif i == 2:
            beta = 1/2 * (c*alpha) * (c*alpha)
            alpha = 1/(d - beta/alpha)
            p = z + beta * p
        else:
            beta = (c*alpha/2) * (c*alpha/2)
            alpha = 1/(d - beta/alpha)
            p = z + beta * p

        new_x = x + alpha * p
        r = b - A @ new_x

        norm1 = np.linalg.norm(x - new_x)
        norm2 = np.linalg.norm(A @ new_x - b) / np.linalg.norm(b)

        x = new_x

        data['i'].append(i)
        data['norm1'].append(norm1)
        data['norm2'].append(norm2)

        i+= 1
    return x, data

In [61]:
eigvalues = sp.linalg.eigvalsh_tridiagonal(np.diag(A), np.diag(A, k=1))
Lmin, Lmax = np.min(eigvalues), np.max(eigvalues)

x, data = chebyshev_method(A, b, Lmax, Lmin, 1e-14)
data['i']

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

In [62]:
jacobi_method(A, b, 1e-14)[1]['i']

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [12]:
import pandas as pd

xs = list(range(15, -1, -1))
ys = [x - 1 if x != 0 else 0 for x in xs]
xs_bin = [ f"{x:04b}" for x in xs]
ys_bin = [f"{x:04b}" for x in ys]

df = pd.DataFrame()
df['Wejście dziesiętne'] = xs
for i, x in enumerate(['A', 'B', 'C', 'D']):
    df[x] = [int(b[i]) for b in xs_bin]

for i in range(4):
    df[f'O{i+1}'] = [int(b[i]) for b in ys_bin]

df['Wyjście dziesiętne'] = ys
print(df.to_latex(index=False))
df


\begin{tabular}{rrrrrrrrrr}
\toprule
Wejście dziesiętne & A & B & C & D & O1 & O2 & O3 & O4 & Wyjście dziesiętne \\
\midrule
15 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 14 \\
14 & 1 & 1 & 1 & 0 & 1 & 1 & 0 & 1 & 13 \\
13 & 1 & 1 & 0 & 1 & 1 & 1 & 0 & 0 & 12 \\
12 & 1 & 1 & 0 & 0 & 1 & 0 & 1 & 1 & 11 \\
11 & 1 & 0 & 1 & 1 & 1 & 0 & 1 & 0 & 10 \\
10 & 1 & 0 & 1 & 0 & 1 & 0 & 0 & 1 & 9 \\
9 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 0 & 8 \\
8 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 7 \\
7 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 0 & 6 \\
6 & 0 & 1 & 1 & 0 & 0 & 1 & 0 & 1 & 5 \\
5 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 0 & 4 \\
4 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 3 \\
3 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 0 & 2 \\
2 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 \\
1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\bottomrule
\end{tabular}



,Wejście dziesiętne,A,B,C,D,O1,O2,O3,O4,Wyjście dziesiętne
0,15,1,1,1,1,1,1,1,0,14
1,14,1,1,1,0,1,1,0,1,13
2,13,1,1,0,1,1,1,0,0,12
3,12,1,1,0,0,1,0,1,1,11
4,11,1,0,1,1,1,0,1,0,10
5,10,1,0,1,0,1,0,0,1,9
6,9,1,0,0,1,1,0,0,0,8
7,8,1,0,0,0,0,1,1,1,7
8,7,0,1,1,1,0,1,1,0,6
9,6,0,1,1,0,0,1,0,1,5


In [9]:
output = 'O4'
tabular = df.to_latex(index=False, columns=['A', 'B', 'C', 'D', output])
template = tabular.replace("tabular", "smalltabular")
print(template)

\begin{smalltabular}{rrrrr}
\toprule
A & B & C & D & O4 \\
\midrule
1 & 1 & 1 & 1 & 0 \\
1 & 1 & 1 & 0 & 1 \\
1 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 0 & 1 \\
1 & 0 & 1 & 1 & 0 \\
1 & 0 & 1 & 0 & 1 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 1 \\
0 & 1 & 1 & 1 & 0 \\
0 & 1 & 1 & 0 & 1 \\
0 & 1 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 \\
0 & 0 & 1 & 1 & 0 \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\bottomrule
\end{smalltabular}



In [24]:
import pandas as pd

xs = list(range(15, -1, -1)) 
xs_bin = [ f"{x:04b}" for x in xs]

df_rs = pd.DataFrame()
df_rs['SET'] = [1] * 16



for i, x in enumerate(['A', 'B', 'C', 'D']):
    df_rs[x] = [int(b[i]) for b in xs_bin]

for i, x in enumerate(['A', 'B', 'C', 'D']):
    df_rs[f'S{i+1}'] = [ 1 if x == 1 else 1  for x in df['A'] ]
    df_rs[f'R{i+1}'] = [ 0 if x == 1 else 1  for x in df['A'] ]



print(df_rs.to_latex(index=False))


\begin{tabular}{rrrrrrrrrrrrr}
\toprule
SET & A & B & C & D & S1 & R1 & S2 & R2 & S3 & R3 & S4 & R4 \\
\midrule
1 & 1 & 1 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 1 & 0 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 0 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 1 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
\bottomrule
\end{tabular}

